# Predictive Maintenance

```SQL
DROP TABLE IF EXISTS maintenance.errors;
DROP TABLE IF EXISTS maintenance.failures;
DROP TABLE IF EXISTS maintenance.maintenance;
DROP TABLE IF EXISTS maintenance.telemetry;
DROP TABLE IF EXISTS maintenance.machines;

CREATE SCHEMA maintenance;

-- 1. Máquinas
CREATE TABLE maintenance.machines (
    machineID INT PRIMARY KEY,
    model VARCHAR(20) NOT NULL,
    age INT NOT NULL
);

-- 2. Telemetría
CREATE TABLE maintenance.telemetry (
    datetime TIMESTAMP NOT NULL,
    machineID INT NOT NULL,
    volt FLOAT,
    rotate FLOAT,
    pressure FLOAT,
    vibration FLOAT,
    PRIMARY KEY (datetime, machineID),
    CONSTRAINT fk_machine_telemetry FOREIGN KEY (machineID) REFERENCES maintenance.machines(machineID)
);

-- 3. Errores
CREATE TABLE maintenance.errors (
    datetime TIMESTAMP NOT NULL,
    machineID INT NOT NULL,
    errorID VARCHAR(20) NOT NULL,
    CONSTRAINT fk_machine_errors FOREIGN KEY (machineID) REFERENCES maintenance.machines(machineID)
);

-- 4. Mantenimiento
CREATE TABLE maintenance.maintenance (
    datetime TIMESTAMP NOT NULL,
    machineID INT NOT NULL,
    comp VARCHAR(20) NOT NULL,
    CONSTRAINT fk_machine_maint FOREIGN KEY (machineID) REFERENCES maintenance.machines(machineID)
);

-- 5. Fallas
CREATE TABLE maintenance.failures (
    datetime TIMESTAMP NOT NULL,
    machineID INT NOT NULL,
    failure VARCHAR(20) NOT NULL,
    CONSTRAINT fk_machine_failures FOREIGN KEY (machineID) REFERENCES maintenance.machines(machineID)
);

-- 6. Índices (Deben apuntar al nuevo schema)
CREATE INDEX idx_telemetry_machine ON maintenance.telemetry(machineID);
CREATE INDEX idx_errors_machine ON maintenance.errors(machineID);
```

# 🏗️ Arquitectura y Despliegue – Plataforma de Mantenimiento Predictivo

---

## 1. 🧩 Definición de la Arquitectura

Se diseñó una arquitectura basada en **microservicios**, contenida dentro de un **ecosistema Docker**.

### Componentes principales

- **🗄️ Contenedor de Base de Datos**
  - Motor: **PostgreSQL 15**
  - Aislado del host, con el puerto **5443** expuesto para administración externa.

- **🖥️ Contenedor de Aplicación**
  - Aplicación **Python** ejecutando un **Dashboard con Dash (Plotly) + Flask**
  - Servido mediante **Gunicorn + Uvicorn** para entornos productivos.

- **🌐 Red (Network)**
  - Red virtual interna de Docker.
  - La aplicación se conecta a la DB usando el **nombre del servicio** (`maintenance_db`) en lugar de direcciones IP.

---

## 2. 📐 Construcción del “Plano” (Docker & Configuración)

Se crearon los archivos maestros para automatizar la infraestructura completa.

### Archivos clave

- **`docker-compose.yml`**
  - Definición de servicios.
  - Configuración de **volúmenes persistentes** (los datos sobreviven reinicios).
  - Variables de entorno (`POSTGRES_USER`, `DB_HOST`, etc.).

- **`.env`**
  - Centralización de credenciales y parámetros sensibles.
  - Permite un código **seguro, modular y portable**.

- **`Dockerfile`**
  - Imagen base: `python:3.11-slim`
  - Instalación de dependencias del sistema (`libpq-dev` para PostgreSQL).
  - Preparación del entorno de trabajo en `/app`.

---

## 3. 🧠 Modelado de Datos (SQLAlchemy 2.0)

Traducción de los archivos **CSV** a modelos de base de datos relacionales.

### Estructura

- **`src/database/base.py`**
  - Definición de la clase base.
  - Configuración del **engine** y la conexión.

- **Modelos (`src/models/`)**
  - `Machine`
  - `Telemetry`
  - `Error`
  - `Maint`
  - `Failure`

### ✅ Desafío superado
- Adaptación al tipado moderno de **SQLAlchemy 2.0** usando `Mapped[tipo]`.
- Eliminación de errores de anotación en **Python 3.14**.

---

## 4. 🔌 El Puente de Conexión (DataGrip)

Configuración de la herramienta profesional para inspeccionar la base de datos dentro del contenedor.

### Configuración clave

- Mapeo de puertos:
  - **Host:** `5443`
  - **Contenedor:** `5432`

### ✅ Desafío superado
- Corrección de la URL de conexión JDBC:
  - Eliminación de los corchetes IPv6 (`[...]`).
  - Uso explícito de `127.0.0.1` para compatibilidad.

- Creación manual del esquema **`maintenance`** para organizar las tablas.

---

## 5. ⚙️ El Motor de Ingesta (`ingestion.py`)

Script inteligente para poblar la base de datos.

### Funcionalidad

- Lectura de archivos **CSV locales**.
- Procesamiento con **Pandas**.
- Inserción eficiente usando **SQLAlchemy**.

### Resultados

- 📊 **876.100 registros de telemetría** cargados exitosamente.
- Inserción completa de máquinas, errores y fallas.
- Verificación de:
  - Llaves foráneas.
  - Tipos de datos (incluyendo `datetime`).

---

## 6. 🚀 Despliegue del Dashboard

Puesta en marcha de la aplicación.

### Detalles del despliegue

- **Imagen Docker:** `predictivemaintenance-app`
- **Puertos:**
  - Interno: `8080`
  - Externo: `8050`

- **Servidor de aplicación**
  - **Gunicorn** con workers **Uvicorn**
  - Optimización de concurrencia y tiempos de respuesta.

---

## ✅ Estado Actual del Sistema

- ✅ Base de datos saludable y completamente poblada.
- ✅ Contenedores corriendo y comunicándose correctamente.
- ✅ Infraestructura persistente: los datos sobreviven reinicios del sistema.

####################################################################################
####################################################################################
####################################################################################

## ⚙️ FastAPI — El Backend / El Motor

FastAPI actúa como el **núcleo del sistema**, encargado de la lógica, los datos y la seguridad.

### Funciones principales

- **🗄️ Gestión de Datos**  
  Se comunica directamente con la base de datos **PostgreSQL** para leer y escribir información.

- **🔐 Seguridad**  
  Permite implementar autenticación y autorización (por ejemplo, proteger el dashboard con usuario y contraseña).

- **🔌 API**  
  Las máquinas reales pueden enviar datos de **temperatura**, **vibración** u otras variables directamente a FastAPI mediante endpoints REST.

- **📘 Documentación Automática**  
  Accediendo a `http://localhost:8050/docs`, FastAPI genera automáticamente un **manual interactivo** de la API (Swagger UI).

---

## 📊 Dash + Plotly — El Frontend / La Cabina de Control

Dash es la capa visual del sistema: todo lo que el usuario ve e interactúa.

### Funciones principales

- **🖥️ Interfaz**  
  Renderiza gráficos, indicadores de alerta, tablas y paneles de control.

- **🧠 Interactividad**  
  Cuando el usuario selecciona una máquina en un *dropdown* o ajusta un filtro, Dash recalcula y actualiza los gráficos en tiempo real.

- **🎨 Presentación**  
  - **Plotly**: Gráficos interactivos y visualmente atractivos.  
  - **Bootstrap**: Botones, colores y layouts con apariencia profesional y responsiva.

####################################################################################
####################################################################################
####################################################################################

## 4. 🎨 Construcción de la Interfaz Visual (Layout)

El diseño del dashboard no se limita a que “se vea bien”; debe ser **funcional, legible y responsivo**.  
Para esto se utiliza **Dash Bootstrap Components (DBC)**, que permite aprovechar el **sistema de rejilla (Grid System)** de Bootstrap dentro de Dash.

### 📐 Estructura de Rejilla (Grid System)

- La pantalla se divide en **12 columnas imaginarias**.
- **Selector de máquina**
  - `lg=3` → ocupa el **25% del ancho** en pantallas grandes.
- **Gráfico y tabla**
  - `lg=9` → ocupa el **75% del ancho** restante.
- **Responsividad**
  - Con `xs=12` se indica:
    > “En pantallas pequeñas (móvil), apila todos los componentes verticalmente para mantener la legibilidad”.

---

### 📊 Componentes de Visualización

- **`dcc.Graph`**
  - No es una imagen estática.
  - Es un objeto **Plotly interactivo** que permite:
    - Zoom
    - Aislar variables (clic en la leyenda)
    - Ver valores exactos al pasar el cursor (*hover*).

- **`dash_table.DataTable`**
  - Configurada en modo **lista** (`style_as_list_view=True`).
  - Se eliminan líneas verticales innecesarias.
  - El foco visual queda en la **fecha** y el **tipo de error**.

---

### 🧱 Contenedores de Estilo (Cards)

- Se utilizan **`dbc.Card`** para separar visualmente controles y datos.
- El parámetro `shadow-sm` añade una sombra sutil.
- Combinado con `bg-light`, se logra una sensación moderna de **capas superpuestas**, mejorando la experiencia visual.

---

## 5. ⚡ Lógica de Interactividad (Callbacks)

Aquí entra en juego la **programación reactiva**.  
El backend del dashboard está constantemente **escuchando cambios** del usuario.

### 🎯 El Disparador (Input)

- Todo comienza cuando cambia:
  ```python
  Input('machine-selector', 'value')